<H2>LLM using Ollama open source models

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = "llama3.2:1b", temperature = 1.8, num_predict = 15)
llm.invoke("What is a Tesla by elon musk, which are the models available")

AIMessage(content='A Tesla is a line of electric vehicles (EVs) designed and manufactured', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2026-01-20T06:43:04.2523188Z', 'done': True, 'done_reason': 'length', 'total_duration': 4941047200, 'load_duration': 4795705500, 'prompt_eval_count': 40, 'prompt_eval_duration': 14521000, 'eval_count': 15, 'eval_duration': 116951300, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019bda24-6e6d-7820-9402-828a36842924-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 40, 'output_tokens': 15, 'total_tokens': 55})

<H2>LLM from Cloud using GroqCloud

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", max_tokens=15)
llm.invoke("What is a Tesla by elon musk, which are the models available")

AIMessage(content='A Tesla is an electric vehicle (EV) manufactured by Tesla, Inc.,', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 50, 'total_tokens': 65, 'completion_time': 0.050747586, 'completion_tokens_details': None, 'prompt_time': 0.002674907, 'prompt_tokens_details': None, 'queue_time': 0.060850984, 'total_time': 0.053422493}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'length', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bda4d-8eb5-7091-9b8f-61b6744420b4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 50, 'output_tokens': 15, 'total_tokens': 65})

<h2>Prompt Templates & Chains

In [12]:
from langchain_core.prompts import PromptTemplate

pt = PromptTemplate.from_template("Write two line poem on {topic}")

In [14]:
llm = ChatGroq(model_name = "llama-3.3-70b-versatile")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000024C37878380>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024C37878DD0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [16]:
chain = pt | llm
response = chain.invoke({'topic': 'cats'})
print(response.content)

Here is a two-line poem about cats:
Cats prowl through the night so free, 
Their eyes shine bright with mystery.


<h2>Output Parser

In [17]:
from langchain_groq import ChatGroq 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException

In [18]:
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
prompt = '''
From the below news article, extract revenue and eps in JSON format containing the
following keys: 'revenue_actual', 'revenue_expected', 'eps_actual', 'eps_expected'.

Each value should have a unit such as million or billion 

Only return the valid JSON. No preamble.

Article
=======
{article}

'''

pt = PromptTemplate.from_template(prompt)
pt

PromptTemplate(input_variables=['article'], input_types={}, partial_variables={}, template="\nFrom the below news article, extract revenue and eps in JSON format containing the\nfollowing keys: 'revenue_actual', 'revenue_expected', 'eps_actual', 'eps_expected'.\n\nEach value should have a unit such as million or billion \n\nOnly return the valid JSON. No preamble.\n\nArticle\n=======\n{article}\n\n")

In [28]:
llm = ChatGroq(model_name = "llama-3.3-70b-versatile")
chain = pt | llm

In [29]:
article_text = '''
Here's what the company reported compared with what Wall Street was expecting, based on a survey

Earnings per share: 72 cents, adjusted vs. 58 cents expected
Revenue: $25.18 billion vs. $25.37 billion expected
'''

response = chain.invoke({'article': article_text})
response.content

'{"revenue_actual": "$25.18 billion", "revenue_expected": "$25.37 billion", "eps_actual": "72 cents", "eps_expected": "58 cents"}'

In [30]:
parser = JsonOutputParser()
output_json = parser.parse(response.content)
output_json

{'revenue_actual': '$25.18 billion',
 'revenue_expected': '$25.37 billion',
 'eps_actual': '72 cents',
 'eps_expected': '58 cents'}

In [31]:
output_json['revenue_actual']

'$25.18 billion'